## Waddah Attar Explosion V2 [SHK]

<https://ru.tradingview.com/script/d9IjcYyS-Waddah-Attar-Explosion-V2-SHK/>

>Developed by [@edyatl](https://github.com/edyatl) April 2023 <edyatl@yandex.ru>

In [1]:
# Load Jupyter extension for auto correction coding style based on Black Lib
%load_ext nb_black

<IPython.core.display.Javascript object>

### Original Indicator code

```python
//
// @author LazyBear 
// List of all my indicators: 
// https://docs.google.com/document/d/15AGCufJZ8CIUvwFJ9W-IKns88gkWOKBCvByMEvm5MLo/edit?usp=sharing
//

// Modified for Crypto Market by ShayanKM

study("Waddah Attar Explosion V2 [SHK]", shorttitle="WAE [SHK]")
sensitivity = input(150, title="Sensitivity")
fastLength=input(20, title="FastEMA Length")
slowLength=input(40, title="SlowEMA Length")
channelLength=input(20, title="BB Channel Length")
mult=input(2.0, title="BB Stdev Multiplier")

DEAD_ZONE = nz(rma(tr(true),100)) * 3.7

calc_macd(source, fastLength, slowLength) =>
	fastMA = ema(source, fastLength)
	slowMA = ema(source, slowLength)
	fastMA - slowMA

calc_BBUpper(source, length, mult) => 
	basis = sma(source, length)
	dev = mult * stdev(source, length)
	basis + dev

calc_BBLower(source, length, mult) => 
	basis = sma(source, length)
	dev = mult * stdev(source, length)
	basis - dev

t1 = (calc_macd(close, fastLength, slowLength) - calc_macd(close[1], fastLength, slowLength))*sensitivity

e1 = (calc_BBUpper(close, channelLength, mult) - calc_BBLower(close, channelLength, mult))

trendUp = (t1 >= 0) ? t1 : 0
trendDown = (t1 < 0) ? (-1*t1) : 0

plot(trendUp, style=columns, linewidth=1, color=(trendUp<trendUp[1])?lime:green, transp=45, title="UpTrend")
plot(trendDown, style=columns, linewidth=1, color=(trendDown<trendDown[1])?orange:red, transp=45, title="DownTrend")
plot(e1, style=line, linewidth=2, color=#A0522D, title="ExplosionLine")
plot(DEAD_ZONE, color=blue, linewidth=1, style=cross, title="DeadZoneLine")

```